# selenium으로 진행

In [29]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import psycopg2
from itertools import product
from time import sleep

In [78]:
category = {
    '서비스안내' : 'service'
   ,'대출정보' : 'loans'
   ,'기타' : 'etc'
}

In [3]:
browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [C:\Users\alsoj\.wdm\drivers\chromedriver\win32\103.0.5060.134\chromedriver.exe] found in cache


In [69]:
BASE_URL = 'https://ols.sbiz.or.kr/ols/man/SMAN051M/page.do'
DETAIL_URL = 'https://ols.sbiz.or.kr/ols/man/SMAN052M/page.do?bltwtrSeq='
# https://ols.sbiz.or.kr/ols/man/SMAN052M/page.do?bltwtrSeq=4

In [46]:
browser.get(BASE_URL)

In [8]:
result_table = browser.find_element(by=By.ID, value='resultList')

In [10]:
rows = result_table.find_elements(by=By.TAG_NAME, value='tr')

In [13]:
detail = browser.find_element(by=By.CLASS_NAME, value='board_view')

In [18]:
detail_title = detail.find_element(by=By.CLASS_NAME, value='fl_l').text

In [20]:
detail_infos = detail.find_element(by=By.CLASS_NAME, value='bvt_class').find_elements(by=By.TAG_NAME, value='li')

In [79]:
gb = ''
reg_dt = ''
view_cnt = ''

for detail_info in detail_infos:
    key = detail_info.text
    value = detail_info.text.split(':')[-1]
    if '구분' in key:
        gb = category[value]
    elif '등록일' in key:
        reg_dt = value
    elif '조회수' in key:
        view_cnt= value

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=103.0.5060.134)
Stacktrace:
Backtrace:
	Ordinal0 [0x00415FD3+2187219]
	Ordinal0 [0x003AE6D1+1763025]
	Ordinal0 [0x002C3E78+802424]
	Ordinal0 [0x002C6854+813140]
	Ordinal0 [0x002C6712+812818]
	Ordinal0 [0x002C69A0+813472]
	Ordinal0 [0x002ED6EF+972527]
	Ordinal0 [0x0030CB8C+1100684]
	Ordinal0 [0x002E8394+951188]
	Ordinal0 [0x0030CDA4+1101220]
	Ordinal0 [0x0031CFC2+1167298]
	Ordinal0 [0x0030C9A6+1100198]
	Ordinal0 [0x002E6F80+946048]
	Ordinal0 [0x002E7E76+949878]
	GetHandleVerifier [0x006B90C2+2721218]
	GetHandleVerifier [0x006AAAF0+2662384]
	GetHandleVerifier [0x004A137A+526458]
	GetHandleVerifier [0x004A0416+522518]
	Ordinal0 [0x003B4EAB+1789611]
	Ordinal0 [0x003B97A8+1808296]
	Ordinal0 [0x003B9895+1808533]
	Ordinal0 [0x003C26C1+1844929]
	BaseThreadInitThunk [0x7570FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77807A9E+286]
	RtlGetAppContainerNamedObjec

In [28]:
content = detail.find_element(by=By.ID, value='cntnDiv').text
content

'온라인으로 대리대출 확인서발급 신청 완료 후,\n자금명, 사업자번호, 대표자 변경 등의 사유로 센터 현장접수로 진행해야 할 경우\n붙임 파일을 작성하시어 해당 지역센터로 제출해주시기 바랍니다.'

In [65]:
def get_recent_id(browser):
    browser.get(BASE_URL)
    return int(browser.find_element(by=By.CLASS_NAME, value='nt_01').text)

In [66]:
recent_id = get_recent_id(browser)
max_id = get_max_id()

In [81]:
for i in range(max_id+1, recent_id+1):
    try:
        title = ''
        gb = ''
        reg_dt = ''
        view_cnt = ''    
        content = ''
        url = DETAIL_URL + str(i)
        browser.get(url)

        detail = WebDriverWait(browser, 3).until(EC.presence_of_element_located((By.CLASS_NAME, "board_view")))
#         detail = browser.find_element(by=By.CLASS_NAME, value='board_view')
        title = detail.find_element(by=By.CLASS_NAME, value='fl_l').text

        detail_infos = detail.find_element(by=By.CLASS_NAME, value='bvt_class').find_elements(by=By.TAG_NAME, value='li')
        for detail_info in detail_infos:
            key = detail_info.text
            value = detail_info.text.split(':')[-1]
            if '구분' in key:
                gb = category[value]
            elif '등록일' in key:
                reg_dt = value
            elif '조회수' in key:
                view_cnt= value

        content = detail.find_element(by=By.ID, value='cntnDiv').text
        insert_ols([str(i), title, gb, reg_dt, content, view_cnt, url])
        
    except Exception as e:
        print('없는 페이지 pass' + str(i))
        continue

없는 페이지 pass1
없는 페이지 pass2
없는 페이지 pass3
없는 페이지 pass5
없는 페이지 pass6
없는 페이지 pass7
없는 페이지 pass11
없는 페이지 pass12
없는 페이지 pass13
없는 페이지 pass14
없는 페이지 pass15
없는 페이지 pass16
없는 페이지 pass17
없는 페이지 pass24
없는 페이지 pass26
없는 페이지 pass27
없는 페이지 pass28
없는 페이지 pass29
없는 페이지 pass30
없는 페이지 pass37
없는 페이지 pass48
없는 페이지 pass51
없는 페이지 pass56


In [62]:
def get_max_id():
    
    sql = """
            select COALESCE(MAX(ID),0) 
            from tb_crawling_ols_intrf
            ;
         """
    conn = None
    max_id = 0
    
    try:
        conn = psycopg2.connect(host='58.120.227.138', dbname='kt_jalnagage',user='postgres',password='Wjdrlwjd1!',port=5432)
        cur = conn.cursor()
        
        cur.execute(sql)
        max_id = cur.fetchone()[0]
        
        
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
    
    return max_id

In [80]:
def insert_ols(params):
    
    sql = """INSERT INTO tb_crawling_ols_intrf(id, title, gb, reg_dt, contents, view_cnt, url, interface_dt)
             VALUES(%s, %s, %s, %s, %s, %s, %s, now());"""
    conn = None

    try:
        conn = psycopg2.connect(host='58.120.227.138', dbname='kt_jalnagage',user='postgres',password='Wjdrlwjd1!',port=5432)
        cur = conn.cursor()
        
        cur.execute(sql, params)
        conn.commit()
        cur.close()
        
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()